# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
!pip install feedparser


In [3]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [5]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [7]:
deals = ScrapedDeal.fetch(show_progress=True)

 80%|████████████████████████████████████         | 4/5 [01:50<00:27, 27.72s/it]/Users/arpita/Desktop/A_Omniverse/Cool_Stuff/LLMS/course_agents/llm_engineering/week8/agents/deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()
100%|█████████████████████████████████████████████| 5/5 [02:24<00:00, 28.83s/it]


In [10]:
len(deals)

50

In [12]:
deals[44].describe()

'Title: Tractor Supply Co. Early Black Friday Deals: Up to 50% off + shipping varies\nDetails: Tractor Supply Co. has released some early Black Friday deals, with discounts up to 50% off. We saw gun safes from $300, flannel shirts for $10, pet needs up to 50% off, and more. Besides the released deals, there will be opportunities for more deals all season like the Doorbuster deals going live Saturday, November 1. Shipping varies based on item and location. Shop Now at Tractor Supply Co.\nFeatures: \nURL: https://www.dealnews.com/Tractor-Supply-Co-Early-Black-Friday-Deals-Up-to-50-off-shipping-varies/21779318.html?iref=rss-c196'

In [16]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [18]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [20]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung U7900 Series UN70U7900FFXZA 70" 4K UHD LED Smart TV (2025) for $398 + free shipping
Details: It's the best we've seen for this model and a great price for a 70" 4K TV. Buy Now at Walmart
Features: 4K resolution 4K upscaling 60Hz refresh rate HDMI Model: UN70U7900FFXZA
URL: https://www.dealnews.com/products/Samsung/Samsung-U7900-Series-UN70-U7900-FFXZA-70-4-K

In [22]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [25]:
result = get_recommendations()

In [28]:
len(result.deals)

5

In [32]:
result.deals[1]

Deal(product_description='The Unlocked Apple iPhone 15 Pro Max boasts an impressive 1TB of storage, making it perfect for those who need ample space for apps, photos, and videos. Its cutting-edge camera system enhances photography with stunning detail and clarity. With a sleek design and robust performance, this smartphone is engineered for speed and reliability. Enjoy the benefits of an unlocked device, providing versatility in carrier choices.', price=747.0, url='https://www.dealnews.com/products/Apple/Unlocked-Apple-iPhone-15-Pro-Max-1-TB/495445.html?iref=rss-c142')

In [34]:
from agents.scanner_agent import ScannerAgent

In [36]:
agent = ScannerAgent()
result = agent.scan()

In [42]:
result

DealSelection(deals=[Deal(product_description='The Samsung U7900 Series UN70U7900FFXZA is a stunning 70-inch 4K UHD LED Smart TV that provides breathtaking picture quality with its 4K resolution and upscaling capabilities. It features a 60Hz refresh rate for smooth motion rendering and multiple HDMI ports for easy connectivity with other devices. The sleek design makes it a perfect addition to any modern living space, offering an immersive viewing experience whether you are watching movies, gaming, or streaming your favorite shows.', price=398.0, url='https://www.dealnews.com/products/Samsung/Samsung-U7900-Series-UN70-U7900-FFXZA-70-4-K-UHD-LED-Smart-TV-2025/495446.html?iref=rss-c142'), Deal(product_description='The Refurbished Unlocked Apple iPhone 15 Pro Max comes with a whopping 1TB of storage, perfect for those who require space for apps, photos, and videos. This device is in excellent condition and has been refurbished to meet high standards. It delivers all the high-end features 